In [3]:
import pandas as pd
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp
import threading
import cv2
from src.utils.Util import *
import psutil
import os


In [13]:
def thread_function(data, image_path, directory_path, dim, org):
    blur = cv2.imread(f'{directory_path}{image_path}')
    blur = cv2.resize(blur, dim)
    data.append(
        [image_path, mse(blur, org), rmse(blur, org), psnr(blur, org), uqi(blur, org), msssim(blur, org), ergas(blur, org),
         scc(blur, org), rase(blur, org), sam(blur, org), vifp(blur, org)])

In [16]:
def get_similar_image(reference_image):
    dim = (400, 400)

    ref = cv2.imread(reference_image)
    ref = cv2.resize(ref, dim)

    data = []
    directory_path = '../../../data/Ground Truth/Newspapers/ddd/'
    threads = list()
    for image in progress_bar(os.listdir(directory_path)[:100]):
        if image == reference_image:
            continue
        # while threading.active_count() > 40:
        #     time.sleep(0.01)
        # while psutil.cpu_percent() > 99:
        #     # time.sleep(0.01)
        #     for thread in progress_bar(threads, desc=f"Joining threads:"):
        #         thread.join()
        x = threading.Thread(target=thread_function, args=(data, image, directory_path, dim, ref,))
        x.start()
        threads.append(x)

    for thread in progress_bar(threads, desc='Joining threads:'):
        thread.join()

    columns = ['MSE', 'RMSE', 'PSNR', 'UQI', 'MSSSIM', 'ERGAS', 'SCC', 'RASE', 'SAM', 'VIF']
    df = pd.DataFrame(data,
                      columns=['FILE', 'MSE', 'RMSE', 'PSNR', 'UQI', 'MSSSIM', 'ERGAS', 'SCC', 'RASE', 'SAM', 'VIF'])
    # print(df['SSIM'])
    for column in columns:
        df[column] = df[column] / df[column].abs().max()

    return df

In [17]:
reference_image = '../../images/templates/white.jpg'

df = get_similar_image(reference_image)

  0%|          | 0/100 [00:00<?, ?it/s]

Joining threads::   0%|          | 0/100 [00:02<?, ?it/s]

In [18]:
print(df.head())

           FILE       MSE      RMSE      PSNR       UQI              MSSSIM  \
0  00530984.tif  0.357546  0.597952  0.959308  0.987860  0.688904+0.000000j   
1  00530988.tif  0.352082  0.593365  0.964219  0.987790  0.697388+0.000000j   
2  00530985.tif  0.379712  0.616208  0.940130  0.985425  0.533141+0.000000j   
3  00530991.tif  0.547459  0.739905  0.823466  0.948336  0.562302+0.000000j   
4  00531000.tif  0.447048  0.668616  0.888074  0.964938  0.797168+0.000000j   

      ERGAS       SCC      RASE       SAM       VIF  
0  0.455503 -0.211127  0.454981  0.685381  0.534428  
1  0.450444 -0.754973  0.449811  0.696318  0.513319  
2  0.463368  0.448105  0.462797  0.792813  0.640777  
3  0.610280  0.540283  0.609650  0.843716  0.460446  
4  0.536052 -0.242219  0.536063  0.646586  0.529441  
